## LSTM Models

- Single Layer LSTM
- Multiple Layers LSTM
- LSTM with Attention

## Model Structures

In [13]:
from tensorflow.keras.utils import plot_model
from data import *
from lstm_model import *

model_types = ['single_layer_lstm', 'multi_layer_lstm', 'lstm_attention', 'attention_lstm']
for model_type in model_types:
    model = LSTMModel(model_path=f"./models/{model_type}_trends")
    model.read_model()
    plot_model(model.model, to_file=f'{model_type}.png', show_shapes=True)

## Activation Experiment

Test on 4 activation functions in LSTM, including:

- sigmoid
- tanh
- relu
- linear

In [1]:
import seaborn as sns
import pickle as pkl
from matplotlib import pyplot as plt

from data import *
from lstm_model import *

# Set plotting style
sns.set_style('whitegrid')
sns.set_palette('Set2')


In [ ]:
# Get confirmed cases dataset
dataset, dataset_rolling = get_dataset_confirmed()
# Print loss of different activations
model_type = 'single_layer_lstm'
activations = ['sigmoid', 'tanh', 'linear', 'relu']
for activation in activations:
    configs = {
        'dataset': dataset_rolling,
        'model_path': f"./models/{model_type}_confirmed_{activation}",
        'model_type': model_type,
        'activation': activation,
    }
    model = LSTMModel(**configs)
    model.read_model()
    # model.plot(f'Confirmed Cases Forcasting with Activation: {activation}')
    model.evaluate()

In [ ]:
import pickle

# Print loss of different activations
for activation in ['linear', 'tanh', 'relu']:
    history_path = f"./models/{model_type}_confirmed_{activation}/history.pkl"
    history_file = open(history_path, 'rb')
    history = pickle.load(history_file)

    plt.figure(figsize=(10, 8))
    plt.plot(history['loss'])
    plt.title(f'Model Loss of {activation}')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    # plt.legend(['Loss'], loc='upper left')
    plt.show()

## Google Trends Forecasting Accuracy

In [3]:
# Get trends dataset
dataset, dataset_rolling = get_dataset_trends()
model_types = ['single_layer_lstm', 'multi_layer_lstm', 'lstm_attention', 'attention_lstm']
# model_types = ['lstm_attention', 'attention_lstm']

vaccine_acc = pd.DataFrame(columns=model_types)
covid_acc = pd.DataFrame(columns=model_types)

for model_type in model_types:
    configs = {
        'dataset': dataset_rolling,
        'model_path': f"./models/{model_type}_trends",
        'model_type': model_type,
    }
    model = LSTMModel(**configs)
    model.read_model()
    model.evaluate()
    vaccine_acc[model_type] = model.get_acc()['trends_vaccine'].values
    covid_acc[model_type] = model.get_acc()['trends_covid'].values

13/13 [==============================] - 0s 3ms/step - loss: 0.0036


In [4]:
vaccine_acc

,single_layer_lstm,multi_layer_lstm,lstm_attention,attention_lstm
0,0.764897,0.760600,0.812198,0.985909
1,0.745847,0.741661,0.792178,0.975640
2,0.729086,0.724996,0.774562,0.964812
3,0.717002,0.712981,0.761918,0.958742
4,0.710459,0.706477,0.755107,0.957607
5,0.693996,0.690107,0.737745,0.941524
6,0.683038,0.679211,0.726214,0.932680
7,0.667807,0.664067,0.710135,0.916855
8,0.652147,0.648495,0.693591,0.900531
9,0.645658,0.642042,0.686805,0.893913


In [9]:
df = vaccine_acc.apply(lambda row: row.mean())
df

single_layer_lstm    0.684232
multi_layer_lstm     0.680397
lstm_attention       0.727409
attention_lstm       0.927024
dtype: float64

In [10]:
covid_acc

,single_layer_lstm,multi_layer_lstm,lstm_attention,attention_lstm
0,0.929392,0.966286,0.947337,0.984699
1,0.927270,0.964075,0.944960,0.983525
2,0.923068,0.959701,0.940463,0.979897
3,0.920979,0.957524,0.938093,0.978486
4,0.916836,0.953212,0.933681,0.975068
5,0.910694,0.946822,0.927227,0.970069
6,0.910692,0.946816,0.927034,0.971791
7,0.900638,0.936360,0.916599,0.962810
8,0.888864,0.924116,0.904399,0.952145
9,0.877393,0.912189,0.892474,0.940130


In [12]:
df = covid_acc.apply(lambda row: row.mean())
df

single_layer_lstm    0.901483
multi_layer_lstm     0.937245
lstm_attention       0.917606
attention_lstm       0.962088
dtype: float64

## Confirmed Cases Forecasting Accuracy

In [16]:
# Get confirmed cases dataset
dataset, dataset_rolling = get_dataset_confirmed()
model_types = ['single_layer_lstm', 'multi_layer_lstm', 'lstm_attention', 'attention_lstm']

acc = pd.DataFrame(columns=model_types)

for model_type in model_types:
    configs = {
        'dataset': dataset_rolling,
        'model_path': f"./models/{model_type}_confirmed",
        'model_type': model_type,
    }
    model = LSTMModel(**configs)
    model.read_model()
    model.evaluate()
    acc[model_type] = model.get_acc()['confirmed'].values

13/13 [==============================] - 0s 3ms/step - loss: 2.2534e-04


In [17]:
acc

,single_layer_lstm,multi_layer_lstm,lstm_attention,attention_lstm
0,0.985172,0.963069,0.988704,0.996721
1,0.983471,0.959926,0.985950,0.995735
2,0.981668,0.956792,0.983044,0.994272
3,0.979731,0.953582,0.980020,0.993342
4,0.977791,0.950413,0.977014,0.991359
5,0.975414,0.947142,0.973891,0.989140
6,0.973297,0.943873,0.970732,0.987642
7,0.971055,0.940614,0.967535,0.985531
8,0.968841,0.937358,0.964274,0.983766
9,0.967106,0.934082,0.960959,0.980876


In [18]:
df = acc.apply(lambda row: row.mean())
df

single_layer_lstm    0.973035
multi_layer_lstm     0.943830
lstm_attention       0.970358
attention_lstm       0.986714
dtype: float64